# Демонстрация работы API

In [1]:
text = "Привет мир"

## Способ 1

In [2]:
from gradio_client import Client

def classify_text(text: str) -> str:
    # Создаем клиент для общения с сервером
    client = Client("http://0.0.0.0:7860/")

    # Отправляем текст для классификации
    result = client.predict(
        text=text,
        api_name="/predict"
    )

    # Обрабатываем результат
    if result:
        status = result[0]
        return status

    return "Ошибка классификации"

# Пример использования функции
status = classify_text(text)

print(f"Текст: {text}")
print(f"Статус: {status}")

/home/timo/rep/TextClassifier/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded as API: http://0.0.0.0:7860/ ✔
Текст: Привет мир
Статус: Positive


## Способ 2

In [3]:
import requests

def classify_text(text: str) -> str:
    # URL и заголовки для POST-запроса
    url = 'http://0.0.0.0:7860/gradio_api/call/predict'
    headers = {'Content-Type': 'application/json'}
    data = {"data": [text]}

    # Отправляем POST-запрос для классификации
    response = requests.post(url, json=data, headers=headers)

    # Проверяем успешность ответа
    if response.status_code == 200:
        # Извлекаем EVENT_ID из ответа
        event_id = response.json().get('event_id')

        # Проверяем, что event_id присутствует
        if event_id:
            # Второй запрос с EVENT_ID для получения классификации
            event_url = f'http://0.0.0.0:7860/gradio_api/call/predict/{event_id}'
            event_response = requests.get(event_url)

            # Если второй запрос успешен
            if event_response.status_code == 200:
                for line in event_response.iter_lines():
                    if line:
                        decoded_line = line.decode('utf-8')

                        if 'data: ' in decoded_line:
                            parsed_data = decoded_line.split('data: ')[1]
                            parsed_data = parsed_data.strip('[]').split(', ')

                            # Извлекаем статус
                            status = parsed_data[0].strip('"')
                            return status

    return "Ошибка классификации"

# Пример использования функции
status = classify_text(text)

print(f"Текст: {text}")
print(f"Статус: {status}")

Текст: Привет мир
Статус: Positive
